# 1. Data Ingestion & Preprocessing

In [1]:
!pip install pypdf nltk

In [2]:
!pip install --upgrade numpy pandas scikit-learn

     --------------------------------------- 11.6/11.6 MB 19.9 MB/s eta 0:00:00
     --------------------------------------- 11.0/11.0 MB 38.6 MB/s eta 0:00:00
  Using cached numpy-1.24.4-cp39-cp39-win_amd64.whl (14.9 MB)
  Attempting uninstall: threadpoolctl
    Found existing installation: threadpoolctl 2.2.0
    Uninstalling threadpoolctl-2.2.0:
      Successfully uninstalled threadpoolctl-2.2.0
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.0
    Uninstalling pandas-2.2.0:
      Successfully uninstalled pandas-2.2.0
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.6.0 requires daal==2021.4.0, which is not installed.
faiss-cpu 1.9.0 requires numpy<3.0,>=1.25.0, but you have numpy 1.24.4 which is incompatible.
numba 0.55.1 requires numpy<1.22,>=1.18, but you have numpy 1.24.4 which is incompatible.


In [3]:
from pypdf import PdfReader  
import nltk

# Download required nltk resources
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# Step 3: Load the PDF and extract text
pdf_path = "C:/Users/ACER/Documents/Chicago/Academic/Quarter4/Capstone/Portillo's Inc., Q4 2023 Earnings Call.pdf"
reader = PdfReader(pdf_path)

Ignoring wrong pointing object 2 0 (offset 0)
Ignoring wrong pointing object 3 0 (offset 0)
Ignoring wrong pointing object 37 0 (offset 0)
Ignoring wrong pointing object 38 0 (offset 0)
Ignoring wrong pointing object 39 0 (offset 0)
Ignoring wrong pointing object 40 0 (offset 0)
Ignoring wrong pointing object 41 0 (offset 0)
Ignoring wrong pointing object 42 0 (offset 0)
Ignoring wrong pointing object 43 0 (offset 0)
Ignoring wrong pointing object 44 0 (offset 0)
Ignoring wrong pointing object 45 0 (offset 0)
Ignoring wrong pointing object 46 0 (offset 0)
Ignoring wrong pointing object 47 0 (offset 0)
Ignoring wrong pointing object 48 0 (offset 0)
Ignoring wrong pointing object 49 0 (offset 0)
Ignoring wrong pointing object 50 0 (offset 0)
Ignoring wrong pointing object 51 0 (offset 0)


In [5]:
all_text = ""
for page_num in range(len(reader.pages)):
    page = reader.pages[page_num]
    all_text += page.extract_text()

In [6]:
# Verify the text extraction
print(all_text[:500])

 
COPYRIGHT © 2024 S&P Global Market Intelligence, a division  of S&P Global Inc. All rights reserved
spglobal.com/marketintelligence1
Portillo's Inc. NasdaqGS:PTLO
FQ4 2023 Earnings Call  Transcripts
Tuesday, February 27, 2024 3:00 PM  GMT
S&P Global Market  Intelligence  Estimates
 -FQ4 2023- -FQ1 2024- -FY 2023- -FY 2024-
 CONSENSUS ACTUAL SURPRISE CONSENSUS CONSENSUS ACTUAL SURPRISE CONSENSUS
EPS 
Normalized  0.05 0.22
 340.00 0.05 0.28 0.71
 153.57 0.39
Revenue  
(mm)184.22 187.86
 1.98 178


In [7]:
# Step 4: Preprocess the extracted text
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

In [8]:
# Tokenize the text
tokens = word_tokenize(all_text)

In [9]:
# Convert tokens to lowercase
tokens = [word.lower() for word in tokens]

In [10]:
# Remove stop words and punctuation
punctuation = string.punctuation
stop_words = set(stopwords.words('english'))
clean_tokens = [word for word in tokens if word not in stop_words and word not in punctuation]

In [11]:
# Display cleaned tokens
print(clean_tokens[:50])

['copyright', '©', '2024', 'p', 'global', 'market', 'intelligence', 'division', 'p', 'global', 'inc.', 'rights', 'reserved', 'spglobal.com/marketintelligence1', 'portillo', "'s", 'inc.', 'nasdaqgs', 'ptlo', 'fq4', '2023', 'earnings', 'call', 'transcripts', 'tuesday', 'february', '27', '2024', '3:00', 'pm', 'gmt', 'p', 'global', 'market', 'intelligence', 'estimates', '-fq4', '2023-', '-fq1', '2024-', '-fy', '2023-', '-fy', '2024-', 'consensus', 'actual', 'surprise', 'consensus', 'consensus', 'actual']


# Embedding Financial Documents

In [12]:
!pip install transformers sentence-transformers

In [13]:
from transformers import AutoTokenizer, AutoModel
import torch

# Load the FinBERT model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("yiyanghkust/finbert-tone")
model = AutoModel.from_pretrained("yiyanghkust/finbert-tone")

In [14]:
# Tokenize the text from your document
inputs = tokenizer(all_text, return_tensors="pt", padding=True, truncation=True, max_length=512)

In [15]:
# Get the embeddings from the model
with torch.no_grad():
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)  # Average the embeddings across all tokens

# Display the shape of the embeddings
print(embeddings.shape)

torch.Size([1, 768])


In [16]:
#print the actual embedding for the first document
print(embeddings[0])

tensor([-2.7319e-01, -3.6481e-01, -3.6161e-01, -1.7184e-01,  1.3324e+00,
        -7.2426e-01, -2.7234e-01,  5.9722e-01,  4.0316e-01,  5.3516e-01,
         5.9313e-01, -5.8368e-01, -1.8264e-01, -5.2914e-01,  5.7329e-01,
         4.3494e-01, -8.8477e-01,  1.1830e+00, -3.8145e-02, -4.0847e-01,
         9.3721e-01, -3.3431e-01,  1.6853e+00,  3.2284e-01, -2.8169e-01,
         9.0930e-01, -5.0063e-01,  5.6692e-01,  7.6061e-01, -3.5531e-01,
        -7.0372e-01, -4.2000e-02,  2.5084e-01, -6.3728e-01, -4.2587e-01,
        -9.1050e-01, -1.1567e+00, -1.1945e-01,  4.2141e-01, -2.2326e-01,
         2.9281e-01, -5.8033e-01, -1.1433e-01, -4.0517e-01, -4.3461e-01,
        -7.8652e-01,  2.7294e-01,  4.1155e-01,  1.1652e+00, -2.1159e-03,
         1.5774e+00,  4.7148e-01, -5.6455e-02,  1.0802e+00,  4.8336e-01,
         2.6336e-01,  1.4127e-01, -8.9795e-01,  4.0267e-01, -2.0294e-01,
         7.4118e-01, -1.6872e-01, -5.8445e-01, -2.7740e-02, -3.4315e-02,
         1.1909e+00, -8.2529e-01,  6.3169e-01, -3.0

# Storing and Indexing

In [17]:
!pip install --user faiss-cpu

  Using cached numpy-2.0.2-cp39-cp39-win_amd64.whl (15.9 MB)


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.6.0 requires daal==2021.4.0, which is not installed.
tensorflow-intel 2.16.1 requires numpy<2.0.0,>=1.23.5; python_version <= "3.11", but you have numpy 2.0.2 which is incompatible.
scipy 1.9.1 requires numpy<1.25.0,>=1.18.5, but you have numpy 2.0.2 which is incompatible.
pyarrow 15.0.0 requires numpy<2,>=1.16.6, but you have numpy 2.0.2 which is incompatible.
numba 0.55.1 requires numpy<1.22,>=1.18, but you have numpy 2.0.2 which is incompatible.
langchain 0.0.354 requires numpy<2,>=1, but you have numpy 2.0.2 which is incompatible.
langchain-community 0.0.20 requires numpy<2,>=1, but you have numpy 2.0.2 which is incompatible.


In [18]:
import faiss
import numpy as np

In [19]:
# Step 1: Convert embeddings to a NumPy array (required for FAISS)
embeddings_np = embeddings.detach().numpy()  # Convert PyTorch tensor to NumPy array

# Step 2: Initialize FAISS index
d = embeddings_np.shape[1]  # Dimension of embeddings
index = faiss.IndexFlatL2(d)  # Using L2 (Euclidean) distance

# Step 3: Add embeddings to the FAISS index
index.add(embeddings_np)

# Check the number of vectors in the index
print(f"Number of vectors in the index: {index.ntotal}")

Number of vectors in the index: 1


In [20]:
# Step 4: Create a query embedding (let's use the first document as a query example)
query_embedding = embeddings_np[0:1]

# Step 5: Perform search in the FAISS index
k = 5  # Number of nearest neighbors to retrieve
distances, indices = index.search(query_embedding, k)

# Display the results
print(f"Top {k} nearest neighbors (indices): {indices}")
print(f"Distances: {distances}")

Top 5 nearest neighbors (indices): [[ 0 -1 -1 -1 -1]]
Distances: [[0.0000000e+00 3.4028235e+38 3.4028235e+38 3.4028235e+38 3.4028235e+38]]
